In [2]:
%pip install accelerate -U
%pip install transformers
%pip install datasets
%pip install seqeval

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
import numpy as np
import pandas as pd
from huggingface_hub import notebook_login

This next part is annoying but necessary to run the code. You have to have a hugging face account and then go [here](https://huggingface.co/settings/tokens), to get a token.

In [4]:
notebook_login()

In [5]:
label_all_tokens = True # dw about it

# Loading the data

In [6]:
def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [7]:
train_data = read_iob2_file("en_ewt-ud-train.iob2")
dev_data = read_iob2_file("en_ewt-ud-dev.iob2")
test_data = read_iob2_file("en_ewt-ud-test-masked.iob2")
print(train_data)


          id                                              words   
0          0             [Where, in, the, world, is, Iguazu, ?]  \
1          1                                    [Iguazu, Falls]   
2          2  [Widely, considered, to, be, one, of, the, mos...   
3          3  [The, centre, of, the, falls, is, known, local...   
4          4  [With, the, sun, shinning, down, on, the, spra...   
...      ...                                                ...   
12538  12538  [Even, a, little, time, spent, receiving, whol...   
12539  12539  [S., and, I, have, an, acquaintance, who, has,...   
12540  12540  [The, urge, to, protect, and, gather, them, al...   
12541  12541              [Take, care, ,, my, friend, ,, Linda]   
12542  12542  [I, 'm, sorry, to, say, Elena, 's, story, has,...   

                                                    tags  
0                              [O, O, O, O, O, B-LOC, O]  
1                                         [B-LOC, I-LOC]  
2      [O, O, O, O

## Tag/index dictionary

In [8]:
class Vocab():
    def __init__(self, pad_unk='<PAD>'):
        self.pad_unk = pad_unk
        self.word2idx = {}
        self.idx2word = []

    def getIdx(self, word, add=False):
        if word is None or word == self.pad_unk:
            return None
        if word not in self.word2idx:
            if add:
                idx = len(self.idx2word)
                self.word2idx[word] = idx
                self.idx2word.append(word)
                return idx
            else:
                return None
        return self.word2idx[word]

    def getWord(self, idx):
        return self.idx2word[idx]

label_indices = Vocab()
tags_column = train_data["tags"]

for tags in tags_column:
    for tag in tags:
        label_indices.getIdx(tag, add=True)

print(label_indices.word2idx)

{'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'I-PER': 6}


Here I add new columns to the data that have the translated tags

In [9]:
train_data['tag_idx'] = train_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])
dev_data['tag_idx'] = dev_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])
test_data['tag_idx'] = test_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])

model_checkpoint = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=True)

train_data

,id,words,tags,tag_idx
0,0,"[Where, in, the, world, is, Iguazu, ?]","[O, O, O, O, O, B-LOC, O]","[0, 0, 0, 0, 0, 1, 0]"
1,1,"[Iguazu, Falls]","[B-LOC, I-LOC]","[1, 2]"
2,2,"[Widely, considered, to, be, one, of, the, mos...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[The, centre, of, the, falls, is, known, local...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[With, the, sun, shinning, down, on, the, spra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
12538,12538,"[Even, a, little, time, spent, receiving, whol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12539,12539,"[S., and, I, have, an, acquaintance, who, has,...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12540,12540,"[The, urge, to, protect, and, gather, them, al...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12541,12541,"[Take, care, ,, my, friend, ,, Linda]","[O, O, O, O, O, O, B-PER]","[0, 0, 0, 0, 0, 0, 3]"


## Tokenization

In [10]:
def tokenize_and_align_labels(dataset, word_column, tag_column, tokenizer):
    tokenized_inputs = tokenizer(dataset[word_column].tolist(), truncation=True, is_split_into_words=True, padding = True)

    labels = []
    for i, label in enumerate(dataset[tag_column]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs.data

In [11]:
train_data

,id,words,tags,tag_idx
0,0,"[Where, in, the, world, is, Iguazu, ?]","[O, O, O, O, O, B-LOC, O]","[0, 0, 0, 0, 0, 1, 0]"
1,1,"[Iguazu, Falls]","[B-LOC, I-LOC]","[1, 2]"
2,2,"[Widely, considered, to, be, one, of, the, mos...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[The, centre, of, the, falls, is, known, local...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[With, the, sun, shinning, down, on, the, spra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
12538,12538,"[Even, a, little, time, spent, receiving, whol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12539,12539,"[S., and, I, have, an, acquaintance, who, has,...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12540,12540,"[The, urge, to, protect, and, gather, them, al...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12541,12541,"[Take, care, ,, my, friend, ,, Linda]","[O, O, O, O, O, O, B-PER]","[0, 0, 0, 0, 0, 0, 3]"


In [12]:
test_data

,id,words,tags,tag_idx
0,0,"[What, is, this, Miramar, ?]","[O, O, O, O, O]","[0, 0, 0, 0, 0]"
1,1,"[It, is, a, place, in, Argentina, lol]","[O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0]"
2,2,"[what, is, a, good, slogan, for, an, Argentini...","[O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"["", In, Argentina, ,, beef, is, revered, ,, re...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,"[Come, see, how, we, continue, this, tradition...","[O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
2072,2072,"[The, radical, Shiite, Sadr, movement, issued,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2073,2073,"[The, statement, said, ,, "", We, direct, an, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2074,2074,"[It, called, a, "", ploy, "", the, assertaion, t...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2075,2075,"[The, convergence, of, views, among, the, more...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
tokenized_data = tokenize_and_align_labels(train_data, "words", "tag_idx", tokenizer)
tokenized_dev_data = tokenize_and_align_labels(dev_data, "words", "tag_idx", tokenizer)
tokenized_test_data = tokenize_and_align_labels(test_data, "words", "tag_idx", tokenizer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Model implementation

In [14]:
label_list = label_indices.idx2word
batch_size = 16

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast) # verifies the tokenizers compatibility with hugging face

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model_name = model_checkpoint.split("/")[-1]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

C:\Users\obe\AppData\Local\Temp\ipykernel_19000\3675664793.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
c:\Users\obe\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [16]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Training

In [17]:
# turning the data into datasetdicts, to make them compatible with the trainer (otherwise they can't be indexed)
train_dataset = Dataset.from_dict({
    'id': range(len(tokenized_data['input_ids'])),
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask'],
    'labels': tokenized_data['labels']
})

dev_dataset = Dataset.from_dict({
    'id': range(len(tokenized_dev_data['input_ids'])),
    'input_ids': tokenized_dev_data['input_ids'],
    'attention_mask': tokenized_dev_data['attention_mask'],
    'labels': tokenized_dev_data['labels']
})

test_dataset = Dataset.from_dict({
    'id': range(len(tokenized_test_data['input_ids'])),
    'input_ids': tokenized_test_data['input_ids'],
    'attention_mask': tokenized_test_data['attention_mask'],
    'labels': tokenized_test_data['labels']
})

In [ ]:
#train_dataset_sm = train_dataset.select(range(50))

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

c:\Users\obe\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

  0%|          | 0/2352 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.save_model("baseline")

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

## Evaluation

In [ ]:
trainer.evaluate()

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.0058501958847046,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.9286285485805678,
 'eval_runtime': 18.3963,
 'eval_samples_per_second': 108.772,
 'eval_steps_per_second': 6.849,
 'epoch': 3.0}

In [ ]:
# evaluating using dev data
dev_dataset_new = Dataset.from_dict({
    'input_ids': dev_dataset['input_ids'],
    'attention_mask': dev_dataset['attention_mask'],
    'labels': dev_dataset['labels']
})

test_dataset_new = Dataset.from_dict({
    'input_ids': test_dataset['input_ids'],
    'attention_mask': test_dataset['attention_mask'],
    'labels': test_dataset['labels']
})

predictions, labels, _ = trainer.predict(test_dataset_new)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/130 [00:00<?, ?it/s]

c:\Users\annam\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.9999672378206599}

In [ ]:
# Print predictions and real values side by side
for i in range(len(true_predictions)):
    print("Example", i+1)
    print("Predicted:", true_predictions[i])
    print("Real:", true_labels[i])
    print()

Example 1
Predicted: ['O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O']

Example 2
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 3
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 4
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 5
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 6
Predicted: ['O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O']

Example 7
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 8
Predicted: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

# Running saved model on the test data

In [1]:
loaded_model = AutoModelForTokenClassification.from_pretrained("annamariagnat/distilbert-base-uncased")

NameError: name 'AutoModelForTokenClassification' is not defined

In [ ]:
trainer = Trainer(model = loaded_model)

In [ ]:
trainer.predict(test_dataset_new)

  0%|          | 0/260 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[[ 9.39851224e-01, -5.91234863e-02, -4.77879405e-01, ...,
         -3.30111086e-01,  6.54929206e-02, -4.36638355e-01],
        [ 1.19645715e+00,  2.53704160e-01, -3.38683128e-01, ...,
         -3.06159198e-01, -2.23335177e-01, -4.79182184e-01],
        [ 1.27412915e+00,  3.23893309e-01, -1.39831215e-01, ...,
         -2.71672428e-01, -4.27593887e-01, -4.80406314e-01],
        ...,
        [ 1.02889466e+00, -1.56676874e-01, -3.26605625e-02, ...,
         -1.68453559e-01, -2.47836024e-01, -3.18754584e-01],
        [ 1.00539732e+00, -1.41223893e-01, -3.23179625e-02, ...,
         -1.50176480e-01, -2.63923645e-01, -3.01223874e-01],
        [ 1.01300812e+00, -1.17688149e-01, -3.34054120e-02, ...,
         -1.44080043e-01, -2.39311129e-01, -3.36805284e-01]],

       [[ 9.06403124e-01, -7.88913220e-02, -4.52483028e-01, ...,
         -2.30923653e-01,  3.82595658e-02, -3.35883141e-01],
        [ 9.22459602e-01, -5.87317050e-02, -3.50926101e-01, ...,
         

In [ ]:
predictions, labels, _ = trainer.predict(test_dataset_new)
predictions = np.argmax(predictions, axis=2)

  0%|          | 0/260 [00:00<?, ?it/s]

In [ ]:
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
# test_dataset_new = Dataset.from_dict({
#     'input_ids': test_dataset['input_ids'],
#     'attention_mask': test_dataset['attention_mask'],
#     'labels': test_dataset['labels']
# })


test_dataset_new["input_ids"][0]

[101,
 2054,
 2003,
 2023,
 18062,
 7849,
 1029,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [ ]:
x = tokenizer.convert_ids_to_tokens(test_dataset_new["input_ids"][0])

In [ ]:
x

['[CLS]',
 'what',
 'is',
 'this',
 'mira',
 '##mar',
 '?',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',


In [ ]:
true_predictions

[['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O', 'O', 'O'],
 ['O', 'O'],
 ['O', 'O', '

In [ ]:
def un_tok_labs(list_of_labels, list_of_words):
    tokenized_inputs = tokenizer(list_of_words, truncation=True, is_split_into_words=True)
    print(tokenized_inputs)
    labels = []
    for i, label in enumerate(list_of_labels):
        print(label)
        label_copy = label.copy()  # Create a copy of the label list

        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        print(word_ids)
        print(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i]))
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            print("word_idx", word_idx)
            if word_idx is None:  # Only label the first token of a given word.
                continue
            elif word_idx == previous_word_idx:
                label_copy.pop(word_idx)
                continue
            else:
                label_ids.append(label_copy[word_idx])
            previous_word_idx = word_idx 
        labels.append(label_ids)
    return labels

In [ ]:
def save_preds(tok, untok_labs):
    with open("group2_preds", "w", encoding="utf-8") as f: 
        for t,l in zip(tok, untok_labs): 
            for i in range(len(t)): 
                f.write(f"{i+1}\t{t[i]}\t{l[i]}\n")

            f.write("\n")
    return ("File has been saved")

In [ ]:
def read_list_of_words(path):
    """
    read in iob2 file
    
    :param path: path to read from
    :returns: list with sequences of words for each sentence
    """
    data = []
    current_words = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')

            current_words.append(tok[1])
        else:
            if current_words:  # skip empty lines
                data.append(current_words)
            current_words = []

    # Check for the last sentence
    if current_words:
        data.append(current_words)
    
    return data

In [ ]:
test_words = read_list_of_words("en_ewt-ud-test-masked.iob2")

In [ ]:
test_words

[['What', 'is', 'this', 'Miramar', '?'],
 ['It', 'is', 'a', 'place', 'in', 'Argentina', 'lol'],
 ['what',
  'is',
  'a',
  'good',
  'slogan',
  'for',
  'an',
  'Argentinian',
  'restaurant',
  '?'],
 ['"',
  'In',
  'Argentina',
  ',',
  'beef',
  'is',
  'revered',
  ',',
  'respected',
  ',',
  'and',
  'praised',
  '.'],
 ['Come', 'see', 'how', 'we', 'continue', 'this', 'tradition', '.', '"'],
 ['A', 'taste', 'of', 'Argentina', '.'],
 ['Here',
  'are',
  'some',
  'articles',
  'that',
  'discuss',
  'the',
  'details',
  'of',
  'slogan',
  'writing',
  '.'],
 ['Why', 'certain', 'slogans', 'work', 'and', 'why', 'some', 'do', "n't", '.'],
 ['You',
  'will',
  'find',
  'these',
  'helpful',
  'in',
  'writing',
  'a',
  'new',
  'slogan',
  '.'],
 ['A',
  'Look',
  'at',
  'Slogans',
  '-',
  'http://www.small-business-software.net/look-at-slogans.htm'],
 ['Unique',
  'Selling',
  'Proposition',
  '-',
  'http://www.small-business-software.net/unique-selling-proposition.htm'],
 ['

In [ ]:
untok_labs = un_tok_labs(true_predictions, test_words)

{'input_ids': [[101, 2054, 2003, 2023, 18062, 7849, 1029, 102], [101, 2009, 2003, 1037, 2173, 1999, 5619, 8840, 2140, 102], [101, 2054, 2003, 1037, 2204, 14558, 2005, 2019, 23157, 15830, 4825, 1029, 102], [101, 1000, 1999, 5619, 1010, 12486, 2003, 23886, 1010, 9768, 1010, 1998, 5868, 1012, 102], [101, 2272, 2156, 2129, 2057, 3613, 2023, 4535, 1012, 1000, 102], [101, 1037, 5510, 1997, 5619, 1012, 102], [101, 2182, 2024, 2070, 4790, 2008, 6848, 1996, 4751, 1997, 14558, 3015, 1012, 102], [101, 2339, 3056, 14558, 2015, 2147, 1998, 2339, 2070, 2079, 1050, 1005, 1056, 1012, 102], [101, 2017, 2097, 2424, 2122, 14044, 1999, 3015, 1037, 2047, 14558, 1012, 102], [101, 1037, 2298, 2012, 14558, 2015, 1011, 8299, 1024, 1013, 1013, 7479, 1012, 2235, 1011, 2449, 1011, 4007, 1012, 5658, 1013, 2298, 1011, 2012, 1011, 14558, 2015, 1012, 1044, 21246, 102], [101, 4310, 4855, 14848, 1011, 8299, 1024, 1013, 1013, 7479, 1012, 2235, 1011, 2449, 1011, 4007, 1012, 5658, 1013, 4310, 1011, 4855, 1011, 14848, 1012

In [ ]:


def save_preds(filename, tok, untok_labs):
    with open(filename, "w", encoding="utf-8") as f: 
        for t,l in zip(tok, untok_labs): 
            for i in range(len(t)): 
                f.write(f"{i+1}\t{t[i]}\t{l[i]}\n")

            f.write("\n")
    return ("File has been saved")

In [ ]:
filename = "test-anna"

save_preds(filename, test_words, untok_labs)

'File has been saved'